# 转置卷积

卷积层通常会使图像的长宽减少，如果有Poling层的话，那么会更加地减少，如果我们的输出图像的尺度并且输入图像相同(这些任务很常见，图像翻译等任务)。

为了实现以上的目标，采用转置卷积ConvTranspose2d来进行上采样，这是2016年提出的，用于逆转下采样造成的空间尺寸的减小。


In [1]:
import torch as t
import torch.nn as nn
import sys
sys.path.append("../")
from pltutils import *

设步幅为1且没有填充。

假设有 $n_h \times n_w$ 的输入张量和一个 $k_h \times k_w$ 的卷积核，以步幅为1滑动卷积核窗口，每行 $n_w$ 次，每列  $n_h$ 次，共产生了 $n_h n_w$ 个中间结果。

每个中间结果都是一个 $(n_h+k_h -1) \times (n_w + k_w -1)$ 的张量，初始化为0。为了计算每个中间张量，输入张量中的每个元素都要乘以卷积核，从而使 $k_h \times k_w$ 张量替换中间张量的一部分。

In [6]:
# 我们对输入矩阵X和卷积核矩阵K实现基本的转置卷积运算trans_conv。
def trans_conv(X:t.Tensor,K:t.Tensor)->t.Tensor:
    h,w = K.shape
    Y = t.zeros((X.shape[0]+h-1,X.shape[1]+w-1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # 注意这里是+=
            Y[i:i+h,j:j+w] += X[i,j]*K
    return Y

In [7]:
X =t.tensor([[0.,1.],[2.,3.]])
K =t.tensor([[0.,1.],[2.,3.]])
trans_conv(X,K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [8]:
# 当输入X和卷积核K都是四维的张量的时候，我们可以使用高级API来获得相同的效果。
X,K = X.reshape(1,1,2,2,),K.reshape(1,1,2,2)
tconv= nn.ConvTranspose2d(1,1,kernel_size=2,bias=False)
tconv.weight.data=K
tconv.forward(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<SlowConvTranspose2DBackward0>)

Padding, Stride and MultiChannel

In [10]:
"""
将高和宽两侧的填充数指定为1的时候，转置卷积的输出中将删除第一和最后的行和列。
"""
tconv = nn.ConvTranspose2d(1,1,kernel_size=2,padding=1,bias=False)
tconv.weight.data=K
tconv.forward(X)


tensor([[[[4.]]]], grad_fn=<SlowConvTranspose2DBackward0>)

In [11]:
"""
步幅为2的时候会增加长宽
"""
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)


tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<SlowConvTranspose2DBackward0>)

In [12]:
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape


True

# 与矩阵变换的关系

给定输入向量$x$和权重矩阵$W$，卷积的前向传播函数可以通过将其输入与权重矩阵相乘并输出向量$y = Wx$实现，反向传播遵循链式法则$ grad \ xy =W^T$

使用矩阵应该比上面的for循环要快